In [2]:
import requests
import json
import pandas as pd

NOTION_API_KEY = "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"
NOTION_URL = "https://api.notion.com/v1/pages"
PAGE_ID = "15b2ec8473e98007885fe7ec343ff6df"

headers = {
    "Authorization": "Bearer " + NOTION_API_KEY,
    "Content-Type": "application/json", 
    "Notion-Version": "2022-06-28", 
}

In [46]:
#helper
def data_conversion(data: dict):
    response = {}
    has_title = False
    for x in data:
        if data[x] == "title":
            tmp_data = {"title" : {}}
            has_title = True

        elif data[x] == "number":
            tmp_data = {"number": {}}

        elif data[x] == "rich_text":
            tmp_data = {"rich_text": {}}

        elif data[x] == "date":
            tmp_data = {"date": {}}

        elif data[x] == "url":
            tmp_data = {"url": {}}
            

        response.update({x: tmp_data})
    
    if not has_title:
        response.update({"Title": {"title": {}}})
    return {"properties": response}

data = {
    "name" : "title",
    "age" : "number"
}

output = data_conversion(data)
with open("output.json", "w") as f:
    json.dump(output, f, indent=4)

In [47]:
#create
def create_customer(id:str, data: dict):
    payload = {
        "parent" : {"type": "page_id", "page_id": PAGE_ID},
        "title": [{
            "type": "text",
            "text": {"content": id}
        }]
    }
    payload.update(data_conversion(data))
    response = requests.post("https://api.notion.com/v1/databases", headers=headers, json=payload)

    print(response.status_code)
    return response.json()
    
def add_row(data: dict):
    pass

In [45]:
# Define the database payload
payload = {
    "parent": {"type": "page_id", "page_id": PAGE_ID},
    "title": [
        {
            "type": "text",
            "text": {
                "content": "My Next New Database"  # Replace with your database title
            }
        }
    ],
    "properties": {
        "Name": {"title": {}},
        "Status": {
            "select": {
                "options": [
                    {"name": "To Do", "color": "blue"},
                    {"name": "In Progress", "color": "yellow"},
                    {"name": "Done", "color": "green"}
                ]
            }
        },
        "Due Date": {"date": {}},
        "Age": {"number": {}},
        "URL": {"url": {}},
        "Description": {"rich_text": {}}
    }
}

# Send the request
response = requests.post(
    "https://api.notion.com/v1/databases",
    headers=headers,
    json=payload
)

# Handle the response
if response.status_code == 200:
    print("Database created successfully!")
    data = response.json()

    with open("output.json", "w") as f:
        json.dump(data, f, indent=4)

else:
    print(f"Failed to create database. Status code: {response.status_code}")


Database created successfully!


In [41]:
#retrieve
def read_db():
    url = f"https://api.notion.com/v1/blocks/{PAGE_ID}/children"
    response = requests.get(url, headers=headers)

    data = response.json()
    result = data["results"]

    while data["has_more"]:
        next_url = url + f"?start_cursor={data['next_cursor']}"
        response = requests.get(next_url, headers=headers)
        data = response.json()  
        result += data["results"]

    if result == []: 
        return None
    return result

def display_db():
    res = read_db()
    if res == None:
        return None

    for i in range(len(res)):
        print(res[i]["child_database"]["title"])

def display_customer_data(dbID : str):
    url = f"https://api.notion.com/v1/databases/{dbID}/query"
    response = requests.post(url, headers=headers, json={"page_size": 100})

    data = response.json()

    with open("output.json", "w") as f:
        json.dump(data, f, indent=4)

    result = data["results"]

    if result == []: 
        return None

    while data["has_more"]:
        next_url = url + f"?start_cursor={data['next_cursor']}"
        response = requests.post(next_url, headers=headers)
        data = response.json()  
        result += data["results"]

    for x in result:
        for y in x["properties"]:
            if x["properties"][y]["type"] == "title":
                print(x["properties"][y]["title"][0]["text"]["content"], end=", ")
            elif x["properties"][y]["type"] == "number":
                print(x["properties"][y]["number"], end=", ")
            elif x["properties"][y]["type"] == "rich_text":
                print(x["properties"][y]["rich_text"][0]["text"]["content"], end=", ")
            elif x["properties"][y]["type"] == "date":
                print(x["properties"][y]["date"]["start"], end=", ")
            elif x["properties"][y]["type"] == "url":
                print(x["properties"][y]["url"], end=", ")
        print()

    

def get_customer_data(customer_ID : str):
    res = read_db()
    if res == None:
        return None
    
    for i in range(len(res)):
        if res[i]["child_database"]["title"] == customer_ID:
            return display_customer_data(res[i]["id"])
    
    return None
 

In [38]:
#update
def update_customer_db(id : str, data: dict):
    url = f"https://api.notion.com/v1/pages/{id}"
    response = requests.patch(url, headers=headers, data=json.dumps(data))

    return response.json()

def update_data(id: str, data: dict):
    pass

In [39]:
#delete
def delete_customer_db(id : str):
    url = f"https://api.notion.com/v1/pages/{id}"
    response = requests.delete(url, headers=headers)

    return response.json()

In [48]:
#test
data = {
    "name":"title", 
    "age":"number",
    "studentID":"rich_text",
    "Enrollment Date":"date",
    "website":"url"
}

create_customer("haha", data)

200


{'object': 'database',
 'id': '15f2ec84-73e9-8138-a2d9-e04b7f430919',
 'cover': None,
 'icon': None,
 'created_time': '2024-12-17T01:46:00.000Z',
 'created_by': {'object': 'user',
  'id': '6cb31505-6086-41b3-80b7-14a4b13e8943'},
 'last_edited_by': {'object': 'user',
  'id': '6cb31505-6086-41b3-80b7-14a4b13e8943'},
 'last_edited_time': '2024-12-17T01:46:00.000Z',
 'title': [{'type': 'text',
   'text': {'content': 'haha', 'link': None},
   'annotations': {'bold': False,
    'italic': False,
    'strikethrough': False,
    'underline': False,
    'code': False,
    'color': 'default'},
   'plain_text': 'haha',
   'href': None}],
 'description': [],
 'is_inline': False,
 'properties': {'age': {'id': '%3C%3A%7B%7D',
   'name': 'age',
   'type': 'number',
   'number': {'format': 'number'}},
  'studentID': {'id': 'FNo%3F',
   'name': 'studentID',
   'type': 'rich_text',
   'rich_text': {}},
  'Enrollment Date': {'id': 'HszZ',
   'name': 'Enrollment Date',
   'type': 'date',
   'date': {}},
  